<a href="https://colab.research.google.com/github/Twenkid/Colab-Notebooks-AI-ML-CV/blob/main/Whisper_Large_8bit_loading_w_bnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper Large inference in 8-bit mode

For faster and memory efficient inference for large models. Read more about it [here](https://huggingface.co/blog/hf-bitsandbytes-integration)

Compiled by: [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb)

We'll first install the necessary packages. We need ffmpeg to decode `mp3` files from the CV11 dataset and transformers, bnb and accelerate to load the model in 8bit mode.

In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4 && apt update && apt install -y ffmpeg
!pip install --quiet datasets git+https://github.com/huggingface/transformers evaluate huggingface_hub jiwer bitsandbytes accelerate

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [1

Since we will be running inference on CV11 dataset, we'd need to authenticate ourselves (since, CV11 requires accepting its Terms and Conditions)

In [3]:
!git config --global credential.helper store
from huggingface_hub import login

login()

To reduce the memory and time overhead, we'll load the dataset in streaming fashion. During the time of inference we'll stream one data point at a time. This is specially useful for larger datasets.

In [4]:
from datasets import load_dataset

dataset = load_dataset(
    "mozilla-foundation/common_voice_11_0", "en", revision="streaming", split="test", streaming=True, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading the model and processor in 8bit mode with `load_in_8bit=True`

Note: This is the only change you need to make in order for you to run the model in 8bit mode.

In [5]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large", device_map="auto", load_in_8bit=True)
processor = WhisperProcessor.from_pretrained("openai/whisper-large", load_in_8bit=True)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [50]:
model_med = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium", device_map="auto", load_in_8bit=True)
processor_med = WhisperProcessor.from_pretrained("openai/whisper-medium", load_in_8bit=True)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Preprocess the dataset to be sampled at 16KHz, since Whisper expects 16KHz input.

In [57]:
model_small = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", device_map="auto", load_in_8bit=True)
processor_small = WhisperProcessor.from_pretrained("openai/whisper-small", load_in_8bit=True)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from datasets import Audio

dataset = dataset.take(10)

# resample to 16kHz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

Voila! Time to run inference loop!

In [51]:
%%time

device = "cuda" if torch.cuda.is_available() else "cpu"

for data in dataset:
    inputs = processor.feature_extractor(data["audio"]["array"], return_tensors="pt", sampling_rate=16_000).input_features.half().to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
    predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
    text = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True, normalize=False)[0]
    print(text)

Reading metadata...: 16354it [00:00, 32806.33it/s]


KeyboardInterrupt: 

In [8]:
!nvidia-smi

In [ ]:
/content/listenRecord_14_5_2024_16-56-4-367.wav

In [10]:
!whisper "/content/listenRecord_14_5_2024_16-56-4-367.wav" --language Bulgarian --word_timestamps True --model large

In [11]:
!ls


In [14]:
!whisper "/content/listenRecord_14_5_2024_16-56-4-367.wav" --language bg --word_timestamps True --model tiny

In [58]:
#import whisper
a = "/content/listenRecord_14_5_2024_16-56-4-367.wav"
a = "/content/listenRecord_14_5_2024_23-39-6-331.wav"
a = "/content/listenRecord_14_5_2024_23-41-50-719.wav"
#audio = whisper.load_audio(a) #it canbe mp3
#audio = whisper.pad_or_trim(audio)

#audio_dataset = datasets.from_dict({"audio": [a]}).cast_column("audio", Audio())

#audio = audio_dataset[0]["audio"] #Audio.load_audio(a)

from datasets import load_dataset
import soundfile as sf

#dataset = load_dataset("audiofolder", data_dir="/content/")


# OR by specifying the list of files
#dataset = load_dataset("audiofolder", data_files=["path/to/audio_1", "path/to/audio_2", ..., "path/to/audio_n"])
#print(dataset)
#audio = dataset[0]["audio"]
audio, sr = sf.read(a)

print(audio, sr)
print("===")

inputs = processor.feature_extractor(audio, return_tensors="pt", sampling_rate=16_000).input_features.half().to(device)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
text = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True, normalize=False)[0]

In [53]:

print(text)

In [54]:
len(text)

73

In [42]:
print(text)

In [59]:
f = open("t.txt", "at", encoding="utf-8")

In [60]:
f.write("\n"); f.write(text)

73

In [37]:
!ls